## All challenge text is excerpted from https://toadstyle.org/cryptopals/57.txt

```
// ------------------------------------------------------------

57. Diffie-Hellman Revisited: Subgroup-Confinement Attacks

This set is going to focus on elliptic curves. But before we get to
that, we're going to kick things off with some classic Diffie-Hellman.

Trust me, it's gonna make sense later.

Let's get right into it. First, build your typical Diffie-Hellman key
agreement: Alice and Bob exchange public keys and derive the same
shared secret. Then Bob sends Alice some message with a MAC over
it. Easy as pie.

Use these parameters:

    p = 7199773997391911030609999317773941274322764333428698921736339643928346453700085358802973900485592910475480089726140708102474957429903531369589969318716771
    g = 4565356397095740655436854503483826832136106141639563487732438195343690437606117828318042418238184896212352329118608100083187535033402010599512641674644143

The generator g has order q:

    q = 236234353446506858198510045061214171961

"Order" is a new word, but it just means g^q = 1 mod p. You might
notice that q is a prime, just like p. This isn't mere chance: in
fact, we chose q and p together such that q divides p-1 (the order or
size of the group itself) evenly. This guarantees that an element g of
order q will exist. (In fact, there will be q-1 such elements.)

Back to the protocol. Alice and Bob should choose their secret keys as
random integers mod q. There's no point in choosing them mod p; since
g has order q, the numbers will just start repeating after that. You
can prove this to yourself by verifying g^x mod p = g^(x + k*q) mod p
for any x and k.

The rest is the same as before.
```

In [1]:
from itertools import chain, count
from functools import reduce
from operator import mul
from random import randrange
from math import log

from challenge_31 import do_sha256, hmac
from challenge_39 import egcd

In [2]:
p = 7199773997391911030609999317773941274322764333428698921736339643928346453700085358802973900485592910475480089726140708102474957429903531369589969318716771
g = 4565356397095740655436854503483826832136106141639563487732438195343690437606117828318042418238184896212352329118608100083187535033402010599512641674644143
q = 236234353446506858198510045061214171961

In [3]:
def int_to_bytes(n):
    return n.to_bytes(64, 'big')

assert log(p, 2) < 64*8  # correctness check: ensure that our inputs to int_to_bytes() will fit into 64 bytes
assert int_to_bytes(17) == b'\x00'*63 + b'\x11'
print("int_to_bytes(): OK")

int_to_bytes(): OK


In [4]:
def primegen(up_to=None):
    yield 2
    d = {}
    counter = count(3, 2) if up_to is None else range(3, up_to, 2)
    for i in counter:
        l = d.pop(i, None)
        if l:
            for n in l:
                d.setdefault(i+(2*n), []).append(n)
            continue
        yield i
        d.setdefault(3*i, []).append(i)

assert all(p == q for p, q in zip(primegen(), [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]))
print("primegen(): OK")

primegen(): OK


In [5]:
# We'll implement Bob as a coroutine. Our attack code will end up functioning as Alice.

def bob_coro(message):
    """Takes Diffie-Hellman public keys as inputs and yields (message, mac) pairs."""
    x = randrange(0, q)
    x_pub = pow(g, x, p)  # public key - only yielded, not used
    print("Bob: Private key =", x)

    h = (yield x_pub)
    while True:
        secret = int_to_bytes(pow(h, x, p))
        K = do_sha256(secret)
        t = hmac(K, message)
        h = (yield (message, t))


bob = bob_coro(b"crazy flamboyant for the rap enjoyment")
bob_pub = next(bob)  # prints private key for reference; returns public key

Bob: Private key = 41677195427755180666458775778041397189


```
How can we attack this protocol? Remember what we said before about
order: the fact that q divides p-1 guarantees the existence of
elements of order q. What if there are smaller divisors of p-1?

Spoiler alert: there are. I chose j = (p-1) / q to have many small
factors because I want you to be happy. Find them by factoring j,
which is:

    j = 30477252323177606811760882179058908038824640750610513771646768011063128035873508507547741559514324673960576895059570

You don't need to factor it all the way. Just find a bunch of factors
smaller than, say, 2^16. There should be plenty. (Friendly tip: maybe
avoid any repeated factors. They only complicate things.)
```

In [6]:
# gather small nonrepeated factors of j
j = 30477252323177606811760882179058908038824640750610513771646768011063128035873508507547741559514324673960576895059570
j_factors = [p for p in primegen(up_to=2**16) if j % p == 0 and (j // p) % p != 0]

assert reduce(mul, j_factors, 1) > q  # make sure we have enough factors for the chinese remainder theorem

j_factors

[2, 5, 109, 7963, 8539, 20641, 38833, 39341, 46337, 51977, 54319, 57529]

```
Got 'em? Good. Now, we can use these to recover Bob's secret key using
the Pohlig-Hellman algorithm for discrete logarithms. Here's how:

1. Take one of the small factors j. Call it r. We want to find an
   element h of order r. To find it, do:

       h := rand(1, p)^((p-1)/r) mod p

   If h = 1, try again.

2. You're Eve. Send Bob h as your public key. Note that h is not a
   valid public key! There is no x such that h = g^x mod p. But Bob
   doesn't know that.

3. Bob will compute:

       K := h^x mod p

   Where x is his secret key and K is the output shared secret. Bob
   then sends back (m, t), with:

       m := "crazy flamboyant for the rap enjoyment"
       t := MAC(K, m)

4. We (Eve) can't compute K, because h isn't actually a valid public
   key. But we're not licked yet.

   Remember how we saw that g^x starts repeating when x > q? h has the
   same property with r. This means there are only r possible values
   of K that Bob could have generated. We can recover K by doing a
   brute-force search over these values until t = MAC(K, m).

   Now we know Bob's secret key x mod r.

5. Repeat steps 1 through 4 many times. Eventually you will know:

       x = b1 mod r1
       x = b2 mod r2
       x = b3 mod r3
       ...

   Once (r1*r2*...*rn) > q, you'll have enough information to
   reassemble Bob's secret key using the Chinese Remainder Theorem.
```

In [7]:
def find_int_of_order_r(r, p):  # helper function for step 1
    while True:
        h = pow(randrange(2, p), (p-1)//r, p)
        if h != 1:
            assert pow(h, r, p) == 1
            return h

def get_residue(target, r, p, quiet=True):
    if not quiet: print(end=f"r = {r} ... ", flush=True)

    # step 1: randomly search the group for an element h of order r
    h = find_int_of_order_r(r, p)
    while True:
        h = pow(randrange(2, p), (p-1)//r, p)
        if h != 1:
            assert pow(h, r, p) == 1
            break

    # steps 2 and 3: send h, get back a message mac'd by our "shared secret"
    message, t = target.send(h)

    # step 4: recover bob's session secret from t
    for i in range(r):
        secret = int_to_bytes(pow(h, i, p))
        K = do_sha256(secret)
        if hmac(K, message) == t:
            break

    if not quiet: print("Done.")
    return i

# step 5: collect residues of bob's private key
residues = [get_residue(bob, r, p, quiet=False) for r in j_factors]

r = 2 ... Done.
r = 5 ... Done.
r = 109 ... Done.
r = 7963 ... Done.
r = 8539 ... Done.
r = 20641 ... Done.
r = 38833 ... Done.
r = 39341 ... Done.
r = 46337 ... Done.
r = 51977 ... Done.
r = 54319 ... Done.
r = 57529 ... Done.


In [8]:
# apply the Chinese Remainder Theorem to recover bob's private key

def mini_crt(a1, n1, a2, n2):
    m1, m2 = egcd(n1, n2)[1:]
    assert m1*n1 + m2*n2 == 1
    a3 = a1*m2*n2 + a2*m1*n1
    n3 = n1*n2
    return (a3 % n3, n3)

def crt(residues, moduli):
    result = (residues[0], moduli[0])
    for t in zip(residues[1:], moduli[1:]):
        result = mini_crt(*result, *t)
    return result


x, m = crt(residues, j_factors)
assert m > q
bob_pub_derived = pow(g, x, p)
print("Recovered Bob's private key:", x)
print()
print("Bob's public key (derived):", bob_pub_derived)
print("Bob's public key (actual): ", bob_pub)
assert bob_pub_derived == bob_pub
print()
print("It worked!")

Recovered Bob's private key: 41677195427755180666458775778041397189

Bob's public key (derived): 6446650910304953078833171190209883203654010875956642140771278121315739653984267855927900852707437865043904079103198714796716970841872547137169245209535516
Bob's public key (actual):  6446650910304953078833171190209883203654010875956642140771278121315739653984267855927900852707437865043904079103198714796716970841872547137169245209535516

It worked!
